# **FACT TABLE - ORDERS**

### **Data Reading**

In [0]:
df = spark.sql("select * from `databricks-catas`.silver.orders_silver")

df.display()

### dim **table**

In [0]:
df_dimCustomer = spark.sql('select dimCustomerKey, customer_id from `databricks-catas`.gold.dimcustomers')


df_dimProduct = spark.sql('select product_id as DimProductKey, product_id as dim_product_id from `databricks-catas`.gold.dimproducts')

### fact **dataframe**

In [0]:
df_fact = df.join(df_dimCustomer, df["customer_id"] == df_dimCustomer["customer_id"], "left").join(df_dimProduct, df["product_id"] == df_dimProduct["dim_product_id"], "left")

df_fact_new = df_fact.drop("dim_customer_id", "dim_product_id", "customer_id")

In [0]:
df_fact_new.display()

### Upsert on **fact**

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists("`databricks-catas`.gold.Fact_Orders"):
    dlt_obj = DeltaTable.forName(spark, '`databricks-catas`.gold.Fact_Orders')
    dlt_obj.alias("trg").merge(
        df_fact_new.alias("src"),
        "trg.order_id = src.order_id AND trg.dimCustomerKey = src.dimCustomerKey AND trg.DimProductKey = src.DimProductKey")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

else:
    df_fact_new.write.format('delta')\
        .option('path', 'abfss://gold@datalakeadbete.dfs.core.windows.net/Fact_Orders')\
        .saveAsTable('`databricks-catas`.gold.Fact_Orders')

In [0]:
spark.sql("SELECT * FROM `databricks-catas`.gold.Fact_Orders").display()